In [1]:
import gspread
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
service_account = gspread.service_account(filename='./config/expense-tracker-358105-199d116b0a6d.json')
workbook = service_account.open('badminton_tracking')
worksheet = workbook.get_worksheet(0)

In [3]:
df = pd.DataFrame(worksheet.get_all_records()).drop(["Timestamp", "result"], axis=1)

df.columns = ["date", "team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2", "points_team_1", "points_team_2", "venue"]

df['winner'] = np.where(df.points_team_1 > df.points_team_2, 'team_1', 'team_2')
df['margin'] = abs(df.points_team_1 - df.points_team_2)
df['total_points_per_game'] = df["points_team_1"] + df["points_team_2"]

df = df.applymap(lambda x: f'{x}'.lower().strip() if isinstance(x, str) else x)

In [4]:
df['point_bins'] = pd.cut(
    df['total_points_per_game'],
    [0, 30, 35, 40, 45, float("inf")],
    right=False,
    labels=['< 30', '30 - 35', '35 - 40', '40 - 45', "> 45"]
)

In [5]:
players_list = list(np.unique(df[['team_1_player_1', 'team_1_player_2', 'team_2_player_1', 'team_2_player_2']].values))

In [6]:
def get_player_stats(player, df):
    player_matches = df[
        np.where(
            np.logical_or.reduce([df[i] == player for i in ["team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2"]]),
            True,
            False
        )
    ].copy()
    
    player_matches["belongs_to"] = np.where(
        np.logical_or(
            *[player_matches[i] == player for i in ["team_1_player_1", "team_1_player_2"]]
        ),
        'team_1',
        'team_2'
    )
    
    player_matches['player_team_points'] = np.where(
        player_matches["belongs_to"] == 'team_1',
        player_matches["points_team_1"],
        player_matches["points_team_2"]
    )
    
    player_matches['result'] = np.where(player_matches.belongs_to == player_matches.winner, "win", "loss")
    player_matches['is_win'] = np.where(player_matches.result == "win", 1, 0)
    
    return player_matches

In [7]:
team_1 = ['ajay', 'nithin']
team_2 = ['swaroop', 'vatsa']

In [8]:
player_matches = get_player_stats(team_1[0], df)
player_matches['partner'] = np.where(
    player_matches["belongs_to"] == 'team_1',
    np.where(
        player_matches["team_1_player_1"] == team_1[0],
        player_matches["team_1_player_2"],
        player_matches["team_1_player_1"]
    ),
    np.where(
        player_matches["team_2_player_1"] == team_1[0],
        player_matches["team_2_player_2"],
        player_matches["team_2_player_1"]
    ),
)

In [9]:
head_2_head_df = player_matches[np.where(
    player_matches['belongs_to'] == 'team_1',
    np.logical_and.reduce([
        player_matches['partner'] == team_1[1],
        player_matches['team_2_player_1'].isin(team_2),
        player_matches['team_2_player_2'].isin(team_2),
    ]),
    np.logical_and.reduce([
        player_matches['partner'] == team_1[1],
        player_matches['team_1_player_1'].isin(team_2),
        player_matches['team_1_player_2'].isin(team_2),
    ])
)].copy().reset_index()

head_2_head_df["other_team_points"] = head_2_head_df["total_points_per_game"] - head_2_head_df["player_team_points"]

In [10]:
head_2_head_df

,index,date,team_1_player_1,team_1_player_2,team_2_player_1,team_2_player_2,points_team_1,points_team_2,venue,winner,margin,total_points_per_game,point_bins,belongs_to,player_team_points,result,is_win,partner,other_team_points
0,42,1/4/2023,ajay,nithin,swaroop,vatsa,18,21,match point - gublaala,team_2,3,39,35 - 40,team_1,18,loss,0,nithin,21
1,47,1/4/2023,ajay,nithin,swaroop,vatsa,18,21,match point - gublaala,team_2,3,39,35 - 40,team_1,18,loss,0,nithin,21


In [11]:
longest_game = head_2_head_df.iloc[head_2_head_df["total_points_per_game"].idxmax(), :].to_dict()
longest_game

{'index': 42,
 'date': '1/4/2023',
 'team_1_player_1': 'ajay',
 'team_1_player_2': 'nithin',
 'team_2_player_1': 'swaroop',
 'team_2_player_2': 'vatsa',
 'points_team_1': 18,
 'points_team_2': 21,
 'venue': 'match point - gublaala',
 'winner': 'team_2',
 'margin': 3,
 'total_points_per_game': 39,
 'point_bins': '35 - 40',
 'belongs_to': 'team_1',
 'player_team_points': 18,
 'result': 'loss',
 'is_win': 0,
 'partner': 'nithin',
 'other_team_points': 21}

In [19]:
def get_game_result_string(game):
    return f"{game['total_points_per_game']} points: ({game['team_1_player_1']}, {game['team_1_player_2']}) {game['points_team_1']} - {game['points_team_2']} ({game['team_2_player_1']}, {game['team_2_player_2']}) on {game['date']} at {game['venue']}"

head_2_head_stats = {
    "total_games": head_2_head_df.shape[0],
    "team_1_wins": head_2_head_df["is_win"].sum(),
    "average_ppg": head_2_head_df["total_points_per_game"].mean(),
    "average_margin_of_victory": head_2_head_df["margin"].mean(),
    "avg_team1_pts": head_2_head_df["player_team_points"].mean(),
    "avg_team2_pts": head_2_head_df["other_team_points"].mean(),
    "longest_game": get_game_result_string(longest_game)
}

head_2_head_stats["team_2_wins"] = head_2_head_stats["total_games"] - head_2_head_stats["team_1_wins"]

head_2_head_stats["team_1_avg_win_margin"] = round(head_2_head_df[head_2_head_df["is_win"] == 1]["margin"].mean(), 2)
head_2_head_stats["team_2_avg_win_margin"] = round(head_2_head_df[head_2_head_df["is_win"] == 0]["margin"].mean(), 2)

head_2_head_stats["team_1_min_points_in_game"] = head_2_head_df["player_team_points"].min()
head_2_head_stats["team_2_min_points_in_game"] = head_2_head_df["other_team_points"].min()

head_2_head_stats["team_1_largest_win"] = np.nan if head_2_head_stats["team_1_wins"] == 0 else get_game_result_string(head_2_head_df.iloc[head_2_head_df[head_2_head_df["is_win"] == 1]["margin"].idxmax(), :].to_dict())
head_2_head_stats["team_2_largest_win"] = np.nan if head_2_head_stats["team_2_wins"] == 0 else get_game_result_string(head_2_head_df.iloc[head_2_head_df[head_2_head_df["is_win"] == 0]["margin"].idxmax(), :].to_dict())

In [20]:
head_2_head_stats["team_1_games_won_after_deuce"] = np.nan if head_2_head_stats["team_1_wins"] == 0 else head_2_head_df[(head_2_head_df["total_points_per_game"] > 40) & (head_2_head_df["is_win"] == 1)].shape[0]
head_2_head_stats["team_2_games_won_after_deuce"] = np.nan if head_2_head_stats["team_2_wins"] == 0 else head_2_head_df[(head_2_head_df["total_points_per_game"] > 40) & (head_2_head_df["is_win"] == 0)].shape[0]

In [21]:
head_2_head_stats["games_gone_beyond_deuces"] = (head_2_head_df["total_points_per_game"] > 40).sum()

In [22]:
head_2_head_stats

{'total_games': 2,
 'team_1_wins': 0,
 'average_ppg': 39.0,
 'average_margin_of_victory': 3.0,
 'avg_team1_pts': 18.0,
 'avg_team2_pts': 21.0,
 'longest_game': '39 points: (ajay, nithin) 18 - 21 (swaroop, vatsa) on 1/4/2023 at match point - gublaala',
 'team_2_wins': 2,
 'team_1_avg_win_margin': nan,
 'team_2_avg_win_margin': 3.0,
 'team_1_min_points_in_game': 18,
 'team_2_min_points_in_game': 21,
 'team_1_largest_win': nan,
 'team_2_largest_win': '39 points: (ajay, nithin) 18 - 21 (swaroop, vatsa) on 1/4/2023 at match point - gublaala',
 'team_1_games_won_after_deuce': nan,
 'team_2_games_won_after_deuce': 0,
 'games_gone_beyond_deuces': 0}

In [23]:
df2 = pd.DataFrame([head_2_head_stats]).T
df2

,0
total_games,2
team_1_wins,0
average_ppg,39.0
average_margin_of_victory,3.0
avg_team1_pts,18.0
avg_team2_pts,21.0
longest_game,"39 points: (ajay, nithin) 18 - 21 (swaroop, va..."
team_2_wins,2
team_1_avg_win_margin,NaN
team_2_avg_win_margin,3.0


In [24]:
comparision_table_list = [
    ["Wins", "Average points per game", "Average Win Margin", "Minimum Points in a Game", "Games Won post Deuce", "Largest Win"],
    [head_2_head_stats["team_1_wins"], head_2_head_stats["avg_team1_pts"], head_2_head_stats["team_1_avg_win_margin"], head_2_head_stats["team_1_min_points_in_game"], head_2_head_stats["team_1_games_won_after_deuce"], head_2_head_stats["team_1_largest_win"]],
    [head_2_head_stats["team_2_wins"], head_2_head_stats["avg_team2_pts"], head_2_head_stats["team_2_avg_win_margin"], head_2_head_stats["team_2_min_points_in_game"], head_2_head_stats["team_2_games_won_after_deuce"], head_2_head_stats["team_2_largest_win"]]
]

In [25]:
table = go.Table(
    header=dict(
        values=["", " & ".join(team_1), " & ".join(team_2)], 
        align="left", 
        height=40, 
        font=dict(color="white", size=16),
        fill_color="lightslategrey"
    ),
    cells=dict(
        values=comparision_table_list,
        align="left",
        height=60,
        font=dict(size=14),
        fill_color="#eceff1"
    )
)
fig = go.Figure(table)
fig

In [98]:
go.Figure(
    go.Pie(
        labels=[' & '.join(team_1), ' & '.join(team_2)],
        values=[head_2_head_stats["team_1_wins"], head_2_head_stats["team_2_wins"]],
        hole=0.3,
        marker=dict(colors=["#b5de2b", "lightslategrey"])
    ),
)

In [131]:
fig = go.Figure(
    go.Indicator(
        mode="gauge+number+delta",
        value=head_2_head_stats['average_ppg'],
        title="Average Points per Game",
        delta={"reference": 38.3}
    )
)

fig.update_traces(gauge_bar_color="#66bb6a")